In [1]:
# importing packages
import pandas as pd
import re

from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from warnings import warn

In [2]:
# replace variables here.
keywords = input("Enter Field/Job Title: ")
location = input("Enter Location: ")
url =('https://www.linkedin.com/jobs/search?keywords=%s&location=%s&sortBy=DD') % (keywords,location)

#url = "https://www.linkedin.com/jobs/search/?f_TPR=r604800&geoId=101174742&keywords=data%20scientist&location=India&sortBy=DD"
no_of_jobs = 25

Enter Field/Job Title: Data Scientist
Enter Location: India


In [3]:
# this will open up new window with the url provided above 
driver = webdriver.Chrome()
driver.get(url)
sleep(3)
action = ActionChains(driver)

In [4]:
# to show more jobs. Depends on number of jobs selected
i = 2
while i <= (no_of_jobs/25): 
    driver.find_element_by_xpath('/html/body/main/div/section/button').click()
    i = i + 1
    sleep(5)

In [5]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 25 jobs.


In [6]:
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

# for loop for job description and criterias
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(3)
    
    # job description
    jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
    job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
    job_desc.append(job_descs)
    
    # job criteria container below the description
    job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
    all_job_criterias = job_criteria_container.find_all("span", class_='job-criteria__text job-criteria__text--criteria')
    
    # Seniority level
    seniority_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[1]'
    seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    level.append(seniority)
    
    # Employment type
    type_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[2]'
    employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    emp_type.append(employment_type)
    
    # Job function
    function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
    job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    functions.append(job_function)
    
    # Industries
    industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
    industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
    industries.append(industry_type)
    
    x = x+1

In [7]:
# to check if we have all information
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))

25
25
25
25
25
25
25
25
25
25


In [8]:
# creating a dataframe
job_data = pd.DataFrame({'Job ID': job_id,
'Date': post_date,
'Company Name': company_name,
'Post': post_title,
'Location': job_location,
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

# cleaning description column
job_data['Description'] = job_data['Description'].str.replace('\n',' ')

print(job_data.info())
job_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job ID        25 non-null     object
 1   Date          25 non-null     object
 2   Company Name  25 non-null     object
 3   Post          25 non-null     object
 4   Location      25 non-null     object
 5   Description   25 non-null     object
 6   Level         25 non-null     object
 7   Type          25 non-null     object
 8   Function      25 non-null     object
 9   Industry      25 non-null     object
dtypes: object(10)
memory usage: 2.1+ KB
None


,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
0,1876238010,2020-05-29,Crescendo Global,Modeling Expert-Python & Big Data-Mumbai (Auto...,"Mumbai, Maharashtra, India",Modeling job opportunity in Mumbai for a profe...,Mid-Senior level,Full-time,Consulting,Information Technology and Services
1,1876235026,2020-05-29,Expedia Group,Software Development Engineer- II (Big Data),"Gurgaon, Haryana, India",Job Description Expedia Group is looking for ...,Entry level,Full-time,Information Technology,Computer SoftwareInformation Technology and Se...
2,1876226986,2020-05-29,A Reputed e-commerce client of UNISON INTERNAT...,Data Scientist (Gurgaon) CTC 15-30 LPA,"Gurgaon, Haryana, India",Job Title: Data Scientist Reports To: Associa...,Mid-Senior level,Full-time,Analyst,Internet
3,1876229146,2020-05-29,Career Soft Solutions Pvt. Ltd.,Opening for Data Science Engineer,"Chennai, Tamil Nadu, India",Requirement: Data Science Engineer Experience ...,Mid-Senior level,Full-time,Information Technology,Human Resources
4,1883741571,2020-05-29,CogniTensor Technology Private Limited,Data Science Cyber Security Internship in Delh...,"New Delhi, Delhi, India",About CogniTensor Technology Private Limited (...,Internship,Full-time,EngineeringInformation Technology,InternetComputer SoftwareHigher Education


In [38]:
job_container

<ul class="jobs-search__results-list"><li class="result-card job-result-card result-card--with-hover-state job-card__contents--active" data-column="1" data-entity-urn="urn:li:jobPosting:1881362118" data-id="1881362118" data-row="1" data-search-id="344fb53c-1ccf-4804-a9cb-2d1392ea29b2" data-tracking-id="SNkWB5tGSUuTrMiOl6gZ7w=="><a class="result-card__full-card-link" data-tracking-control-name="public_jobs_job-result-card_result-card_full-click" data-tracking-will-navigate="" href="https://in.linkedin.com/jobs/view/viaprom-technologies-senior-data-scientist-machine-learning-data-visualization-8-10-yrs-mumbai-at-viaprom-technologies-pvt-ltd-1881362118?refId=344fb53c-1ccf-4804-a9cb-2d1392ea29b2&amp;position=1&amp;pageNum=0&amp;trk=public_jobs_job-result-card_result-card_full-click"><span class="screen-reader-text">Viaprom Technologies - Senior Data Scientist - Machine Learning/Data Visualization (8-10 yrs) Mumbai</span></a><img alt="Viaprom Technologies Pvt Ltd" aria-hidden="true" class="

In [60]:
 job_titles

'Supply Chain Manager'

In [61]:
company_name

['Viaprom Technologies Pvt Ltd',
 'Sony Research India',
 'SVKM&#39;s Narsee Monjee Institute of Management Studies (NMIMS)',
 'General Mills',
 'General Mills',
 'Viacom18 Media Private Limited',
 'Career Makers',
 'Vision Insight',
 'Time Analytic &amp; Shared Services Pvt. Ltd',
 'ISS | Institutional Shareholder Services',
 'ISS | Institutional Shareholder Services',
 'Epicurean Enterprises Pvt Ltd',
 'Square Panda Inc.',
 'HireTale',
 'Square Panda Inc.',
 'Assort Staffing',
 'Quantsapp Pvt. Ltd.',
 'ImpactGuru',
 'Crescendo Global ',
 'Mactores',
 'Quantsapp Pvt. Ltd.',
 'Square Panda Inc.',
 'Indus OS',
 'Sugarbox Networks pvt ltd',
 'NCR India']